In [1]:
!git clone -b video https://github.com/wisleythewise/custom_diffusers_library.git

Cloning into 'custom_diffusers_library'...
remote: Enumerating objects: 1751, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 1751 (delta 29), reused 38 (delta 14), pack-reused 1693
Receiving objects: 100% (1751/1751), 33.19 MiB | 15.14 MiB/s, done.
Resolving deltas: 100% (655/655), done.
Updating files: 100% (1191/1191), done.


In [1]:
%cd /content/custom_diffusers_library
!pip install -e .
!pip install safetensors

/content/custom_diffusers_library
Obtaining file:///content/custom_diffusers_library
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for diffusers (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.26.0.dev0-0.editable-py3-none-any.whl size=11098 sha256=3247a110f6b8d67fbe54b999caffe3018dd5578074cae77a35f0b4dfe4ffd2d4
  Stored in directory: /tmp/pip-ephem-wheel-cache-uehc4v7j/wheels/f6/06/4f/ee79f1ccb98c1c1d7c14fc34a8cf908c7f73d80d46952161c2
Successfully built diffusers
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.26.0.dev0
    Uninstalling diffusers-0.26.0.dev0:
      Successfully uninstalled diffusers-0.26.0.dev0


In [2]:
!pip install transformers accelerate datasets

In [3]:
import debugpy
import gc
from transformers import CLIPImageProcessor, CLIPTextModel, CLIPTokenizer, CLIPVisionModelWithProjection
from typing import Optional, Tuple, Union
from dataclasses import dataclass
import torch
import torch.nn as nn
from torch.nn import functional as F
from diffusers.models.unets import UNetSpatioTemporalConditionModel
from diffusers import StableVideoDiffusionPipeline
from diffusers.utils import load_image, export_to_video
from diffusers.utils.torch_utils import is_compiled_module, randn_tensor
import torch
import torch.nn as nn
from diffusers.configuration_utils import ConfigMixin, register_to_config
from diffusers.loaders import UNet2DConditionLoadersMixin
from diffusers.utils import BaseOutput, logging
from diffusers.models.attention_processor import CROSS_ATTENTION_PROCESSORS, AttentionProcessor, AttnProcessor
from diffusers.models.embeddings import TimestepEmbedding, Timesteps
from diffusers.models.modeling_utils import ModelMixin
from diffusers.models.unets.unet_3d_blocks import UNetMidBlockSpatioTemporal, get_down_block, get_up_block
from diffusers.models.unets import UNetSpatioTemporalConditionModel
from diffusers.pipelines.stable_video_diffusion.pipeline_stable_video_diffusion_with_controlnet import StableVideoDiffusionPipelineWithControlNet,SpatioTemporalControlNet, CustomConditioningNet, SpatioTemporalControlNetOutput

from diffusers.image_processor import VaeImageProcessor

/content/custom_diffusers_library/src/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/content/custom_diffusers_library/src/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/content/custom_diffusers_library/src/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [4]:
from google.colab import drive

drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
from diffusers import StableVideoDiffusionPipeline
from diffusers import DiffusionPipeline
import torch
checkpoint_path = "/content/drive/MyDrive/checkpoints/model_checkpoint_1235.ckpt"
pipe = StableVideoDiffusionPipeline.from_pretrained(
    "stabilityai/stable-video-diffusion-img2vid", torch_dtype=torch.float16, variant="fp16", ignore_mismatched_sizes=False
)

pipeline = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2")

my_net =  UNetSpatioTemporalConditionModel()
control_net = SpatioTemporalControlNet()
# my_net.load_state_dict(checkpoint['unet_state_dict'])

# using torch
if True:
  checkpoint = torch.load(checkpoint_path)
  my_net = pipe.unet
  control_net.load_state_dict(checkpoint['model_state_dict'])

if False:
  model_path = '/content/drive/MyDrive/checkpoints/checkpoint-15000/diffusion_pytorch_model.safetensors'
  from safetensors.torch import load
  # Load the model
  # Make sure to open the file in binary mode
  with open(model_path, 'rb') as f:
      state_dict = load(f)

  control_net.load_state_dict(state_dict)

pipe_zero_with_controlnet = StableVideoDiffusionPipelineWithControlNet(
    vae = pipe.vae,
    image_encoder = pipe.image_encoder,
    unet=my_net.to(dtype=torch.float16, device=torch.device("cuda")),
    scheduler=pipe.scheduler,
    feature_extractor=pipe.feature_extractor,
    controlnet=control_net.to(dtype=torch.float16, device=torch.device("cuda")),
    tokenizer = pipeline.tokenizer,
    text_encoder = pipeline.text_encoder
)

model_index.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

model.fp16.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

image_encoder/config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

tokenizer/special_tokens_map.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.46G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import json
from diffusers.image_processor import VaeImageProcessor

class DiffusionDataset(Dataset):
    def __init__(self, json_path = None , dataset_huggingface = None):

        if json_path and dataset_huggingface:
            raise ValueError("You can only use one of the two options")

        if json_path:
            with open(json_path, 'r') as f:
                self.data = json.load(f)
            self.using_json = True
        else:
            self.data = dataset_huggingface['train']  # Assuming you want the 'train' split
            self.using_json = False


        self.transform = transforms.Compose([
            transforms.Resize((320, 512)),
            transforms.CenterCrop((320, 512)),
        ])
        self.image_processor = VaeImageProcessor(vae_scale_factor=8)


    def __len__(self):
        # Assuming each set of ground truths represents a separate sample
        return len(self.data['ground_truth'])

    def __getitem__(self, idx):
        # Processing ground truth images

        ground_truth_images = [self.transform(Image.open(path)) for path in self.data['ground_truth'][idx]] \
                                if self.using_json else [self.data['ground_truth'][h] for h in range(14) ]
        ground_truth_images = self.image_processor.preprocess(image = ground_truth_images, height = 320, width = 512)

        prescan_images = [self.transform(Image.open(path)) for path in self.data['prescan_images'][idx]] \
                            if self.using_json else [self.data['prescan_images'][h] for h in range(14)]
        prescan_images = self.image_processor.preprocess(image = prescan_images, height = 320, width = 512)

        # Processing conditioning images set one (assuming RGB, 4 channels after conversion)
        conditioning_images_one = [self.transform(Image.open(path)) for path in self.data['conditioning_images_one'][idx]] \
                                    if self.using_json else [self.data['conditioning_images_one'][h] for h in range(14)]
        conditioning_images_one = self.image_processor.preprocess(image = conditioning_images_one, height = 320, width = 512)

        # Processing conditioning images set two (assuming grayscale, converted to RGB to match dimensions)
        conditioning_images_two = [self.transform(Image.open(path)) for path in self.data['conditioning_images_two'][idx]] \
                                    if self.using_json else [self.data['conditioning_images_two'][h] for h in range(14)]
        conditioning_images_two = self.image_processor.preprocess(image = conditioning_images_two, height = 320, width = 512)

        # Concatenating condition one and two images along the channel dimension
        conditioned_images = [torch.cat((img_one, img_two), dim=0) for img_one, img_two in zip(conditioning_images_one, conditioning_images_two)]

        # Processing reference images (single per scene, matched by index)
        reference_image = self.transform(Image.open(self.data['ground_truth'][idx][0])) \
                        if self.using_json else [self.data['ground_truth'][h] for h in range(14)]

        # Retrieving the corresponding caption
        caption = self.data['caption'][idx][0]



        return {
            "ground_truth": ground_truth_images,
            "conditioning": torch.stack(conditioned_images),
            "caption": caption,
            "reference_image": reference_image,
            "prescan_images": prescan_images
        }

def collate_fn(batch):
    ground_truth = torch.stack([item['ground_truth'] for item in batch])
    conditioning = torch.stack([item['conditioning'] for item in batch])
    prescan_images = torch.stack([item['prescan_images'] for item in batch])
    captions = [item['caption'] for item in batch]  # List of strings, no need to stack
    reference_images = [item['reference_image'] for item in batch]


    return {
        "ground_truth": ground_truth,
        "conditioning": conditioning,
        "caption": captions[0],
        "reference_image": reference_images[0],
        "prescan_images": prescan_images
    }




from datasets import load_dataset
dataset_from_huggingface = load_dataset("JaspervanLeuven/aug_427")
train_dataset = DiffusionDataset(dataset_huggingface=dataset_from_huggingface)

# train_dataset = DiffusionDataset(json_path='./test_jappie.json')

# data = load_from_disk(train_dataset)






In [ ]:
# Assuming `model` is your trained model instance
def print_sum_of_weights_for_zero_initialized_layers(model):
    layers_of_interest = [model.controlnet_mid_block] + list(model.controlnet_down_blocks)
    for i, layer in enumerate(layers_of_interest):
        if isinstance(layer, nn.Conv2d):
            weight_sum = layer.weight.sum().item()
            print(f"Layer {i} weight sum: {weight_sum}")
print_sum_of_weights_for_zero_initialized_layers(control_net)

In [ ]:
train_dataloader = DataLoader(
    train_dataset,
    shuffle=False,
    collate_fn=collate_fn,
    batch_size=1,  # Or your preferred batch size
    num_workers=0,  # Adjust based on your setup
)



batch_zero = batch = train_dataloader.__iter__().__next__()
image = batch_zero["reference_image"][0]
prompt = "A driving scene during the day, with clear weather in boston"
pseudo_sample = batch_zero['conditioning'][0]
# pseudo_sample = torch.rand(pseudo_sample.size())
generator = torch.manual_seed(100)

pipe_zero_with_controlnet.to(dtype=torch.float16, device = torch.device("cuda"))

frames_zero_with_controlnet = pipe_zero_with_controlnet(height=320,width=512, image=image,num_frames = 14, prompt=prompt, conditioning_image = pseudo_sample,  decode_chunk_size=8, generator=generator).frames[0]


In [ ]:
print(batch_zero['conditioning'][0].mean())

In [ ]:
export_to_video(frames_zero_with_controlnet, "Differnet_promt_rainy_night.mp4", fps=7)

In [ ]:
import sys
sys.path.append('/content/custom_diffusers_library')

# Now try importing your custom package
from diffusers import StableVideoDiffusionPipeline


import torch
from diffusers.utils import load_image, export_to_video

pipe = StableVideoDiffusionPipeline.from_pretrained(
    "stabilityai/stable-video-diffusion-img2vid-xt", torch_dtype=torch.float16, variant="fp16", num_frames = 2
)
pipe.enable_model_cpu_offload()



image = load_image("/content/frame0.png")
image = image.resize((64, 64))


generator = torch.manual_seed(42)
frames = pipe(image, decode_chunk_size=8, generator=generator).frames[0]

export_to_video(frames, "car_scene.mp4", fps=7)
